<a href="https://colab.research.google.com/github/PrasunaA/LangChain1/blob/main/LangChain_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install required libraries
!pip install transformers langchain huggingface_hub

In [ ]:
# ✅ Imports
from transformers import pipeline
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import Generation, LLMResult
from langchain_core.pydantic_v1 import Field
from typing import Optional, List, Any
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
# ✅ Load the Hugging Face FLAN-T5 model (no API key required)
generator = pipeline("text2text-generation", model="google/flan-t5-large")

In [4]:
# ✅ Define LangChain-compatible wrapper
class HuggingFacePipelineLLM(LLM):
    pipeline: Any = Field(...)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        result = self.pipeline(prompt, max_length=256, do_sample=True)
        return result[0]["generated_text"]

    def _generate(self, prompt: str, stop: Optional[List[str]] = None) -> LLMResult:
        output_text = self._call(prompt, stop)
        return LLMResult(generations=[[Generation(text=output_text)]])

    @property
    def _llm_type(self) -> str:
        return "custom-hf-pipeline"

In [5]:
# ✅ Wrap the model
wrapped_llm = HuggingFacePipelineLLM(pipeline=generator)

# **Example 2: Summarize a Short News Headline**

In [ ]:
summary_prompt = PromptTemplate.from_template("Summarize this in one sentence: {news}")
summary_chain = LLMChain(llm=wrapped_llm, prompt=summary_prompt)

news_text = " GitHub and GitHub Gist are both valuable tools provided by GitHub for code hosting and collaboration, but they serve different primary purposes. Here's a breakdown of the key differences"
response = summary_chain.invoke({"news": news_text})
print("Summary:\n", response["text"])